# Tidying and Transforming Data
### DAV 5400 Spring 2019 Week 9 Assignment 
__Maayan Raviv__
***


### Question 1
Insert the raw data into CSV file and read from GitHub.

In [6]:
# Question 1
import pandas as pd 
data = pd.read_csv("https://raw.githubusercontent.com/maayan3007/DAV-5400/master/ass9.csv") 

data.head()

,AirLine,Status,LosAngeles,Phoenix,SanDiego,SanFrancisco,Seattle
0,ALASKA,onTime,497,221,212,503,1841
1,ALASKA,delayed,62,12,20,102,305
2,AMWEST,onTime,694,4840,383,320,201
3,AMWEST,delayed,117,415,65,129,61


### Question 2
Converting the raw data to a "tidy", long format.
<br>
* For long format-Cloumns are Airport, Status(onTime/delayed), AirLine(ALASKA/AMWEST), Count.
* For Tidy Format- Add ID cloumn reference-  https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.wide_to_long.html
* Rename columns



In [7]:
# Question 2
# In the long format == tidy, columns are Airport, Status(onTime/delayed), AirLine(ALASKA/AMWEST), count

# Conversion to tidy dataset started in step by step
# Conversion to tidy - step 1: add id coloumn with the intention of using wide_to_long pandas method 
# Ref to method: https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.wide_to_long.html
data["id"] = data.index
data.head()

,AirLine,Status,LosAngeles,Phoenix,SanDiego,SanFrancisco,Seattle,id
0,ALASKA,onTime,497,221,212,503,1841,0
1,ALASKA,delayed,62,12,20,102,305,1
2,AMWEST,onTime,694,4840,383,320,201,2
3,AMWEST,delayed,117,415,65,129,61,3


In [8]:
# Conversion to tidy - step 2: rename columns (In pandas version 0.23, wide_to_long cannot be performed on columns composed by word word. It must be word number format. )
# Ref: https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.wide_to_long.html
airportmapper = {1: "LosAngeles", 2: "Phoenix", 3: "SanDiego", 4: "SanFrancisco", 5: "Seattle"}
columnmapper = {"LosAngeles": "count1", "Phoenix": "count2", "SanDiego": "count3", "SanFrancisco": "count4", "Seattle": "count5"}

data = data.rename(index=str, columns=columnmapper)
data

,AirLine,Status,count1,count2,count3,count4,count5,id
0,ALASKA,onTime,497,221,212,503,1841,0
1,ALASKA,delayed,62,12,20,102,305,1
2,AMWEST,onTime,694,4840,383,320,201,2
3,AMWEST,delayed,117,415,65,129,61,3


In [9]:
# Conversion to tidy - step 3: perform wide_to_long function
data_long = pd.wide_to_long(data, ["count"], i="id", j="Airport")
# Conversion to tidy done
data_long

,,Status,AirLine,count
id,Airport,,,
0,1,onTime,ALASKA,497
1,1,delayed,ALASKA,62
2,1,onTime,AMWEST,694
3,1,delayed,AMWEST,117
0,2,onTime,ALASKA,221
1,2,delayed,ALASKA,12
2,2,onTime,AMWEST,4840
3,2,delayed,AMWEST,415
0,3,onTime,ALASKA,212


In [10]:
# Reset index
data_long = data_long.reset_index()
data_long = data_long.drop(['id'], axis=1)
data_long

,Airport,Status,AirLine,count
0,1,onTime,ALASKA,497
1,1,delayed,ALASKA,62
2,1,onTime,AMWEST,694
3,1,delayed,AMWEST,117
4,2,onTime,ALASKA,221
5,2,delayed,ALASKA,12
6,2,onTime,AMWEST,4840
7,2,delayed,AMWEST,415
8,3,onTime,ALASKA,212
9,3,delayed,ALASKA,20


In [11]:
# Drop unnecessory column id
data_long['Airport'] = data_long['Airport'].map(airportmapper)
data_long
# Answer for Question 2

,Airport,Status,AirLine,count
0,LosAngeles,onTime,ALASKA,497
1,LosAngeles,delayed,ALASKA,62
2,LosAngeles,onTime,AMWEST,694
3,LosAngeles,delayed,AMWEST,117
4,Phoenix,onTime,ALASKA,221
5,Phoenix,delayed,ALASKA,12
6,Phoenix,onTime,AMWEST,4840
7,Phoenix,delayed,AMWEST,415
8,SanDiego,onTime,ALASKA,212
9,SanDiego,delayed,ALASKA,20


### Question 3
Compare the arrival delays for the two airlines.
<br>
We will answer the following questions:
* For each city, which airline had the best on time performance?
* Which airline had the best overall on time performance?

In [12]:
# Question 3
# Q3.1 For each city, which airline had the best on time performance?
data_long_copy = data_long.copy()
data_long_copy = data_long_copy.groupby(['Airport', 'AirLine', 'Status'], as_index=False).sum() # Group by 'Airport', 'AirLine', 'Status' and get the sum of aggregated columns 

data_long_ontime = data_long_copy[data_long_copy['Status']=="onTime"] # Select all records where status is onTime
data_long_ontime = data_long_ontime.rename(index=str, columns={"count": "ontimeCount"}) # Rename the count to ontimeCount
data_long_ontime = data_long_ontime.reset_index(drop=True) # Remove indexing

print(data_long_ontime)

data_long_sum = data_long_copy.groupby(['Airport', 'AirLine'], as_index=False).sum() # Group by 'Airport', 'AirLine' and get the sum of onTime and delayed count sum
data_long_sum = data_long_sum.rename(index=str, columns={"count": "sum"}) # Rename count column name 
data_long_sum = data_long_sum.reset_index(drop=True) # Remove indexing

print('\n\n\n\n', data_long_sum)

merged_dataset = data_long_ontime.join(data_long_sum, rsuffix='_l') # Join both dataframe to have sum and ontimeCount in a single dataframe
merged_dataset = merged_dataset.drop(['Airport_l', 'AirLine_l'], axis=1) # Drop unnecessory columns derived after merging

print('\n\n\n\n', merged_dataset)

# Add column shows performance
merged_dataset['performance'] = merged_dataset.apply(lambda x: x[3]*100/x[4], axis=1) # calc performance and add it in a new column

print('\n\n\n\n', merged_dataset)

print("\nWhich airline perform better in each city(airport).")
merged_dataset = merged_dataset.groupby(['Airport']).apply(lambda dfg: dfg.nlargest(1, 'performance')).reset_index(drop=True) # Group by airport and get the largest record of each in the group based on performance
merged_dataset


        Airport AirLine  Status  ontimeCount
0    LosAngeles  ALASKA  onTime          497
1    LosAngeles  AMWEST  onTime          694
2       Phoenix  ALASKA  onTime          221
3       Phoenix  AMWEST  onTime         4840
4      SanDiego  ALASKA  onTime          212
5      SanDiego  AMWEST  onTime          383
6  SanFrancisco  ALASKA  onTime          503
7  SanFrancisco  AMWEST  onTime          320
8       Seattle  ALASKA  onTime         1841
9       Seattle  AMWEST  onTime          201




         Airport AirLine   sum
0    LosAngeles  ALASKA   559
1    LosAngeles  AMWEST   811
2       Phoenix  ALASKA   233
3       Phoenix  AMWEST  5255
4      SanDiego  ALASKA   232
5      SanDiego  AMWEST   448
6  SanFrancisco  ALASKA   605
7  SanFrancisco  AMWEST   449
8       Seattle  ALASKA  2146
9       Seattle  AMWEST   262




         Airport AirLine  Status  ontimeCount   sum
0    LosAngeles  ALASKA  onTime          497   559
1    LosAngeles  AMWEST  onTime          694   811
2       Phoe

,Airport,AirLine,Status,ontimeCount,sum,performance
0,LosAngeles,ALASKA,onTime,497,559,88.908766
1,Phoenix,ALASKA,onTime,221,233,94.849785
2,SanDiego,ALASKA,onTime,212,232,91.379310
3,SanFrancisco,ALASKA,onTime,503,605,83.140496
4,Seattle,ALASKA,onTime,1841,2146,85.787512


In [13]:
# Q3.2 Which airline had the best overall on time performance? (Logic is similar to the above case. )
data_long_copy = data_long.copy()
data_long_copy = data_long_copy.groupby(['AirLine', 'Status'], as_index=False).sum()

data_ontime = data_long_copy[data_long_copy['Status']=='onTime']
data_ontime = data_ontime.rename(index=str, columns={"count": "ontimeCount"})
data_ontime = data_ontime.reset_index(drop=True)

data_sum = data_long_copy.groupby(['AirLine'], as_index=False).sum()
data_sum = data_sum.rename(index=str, columns={"count": "sum"})
data_sum = data_sum.reset_index(drop=True)

merged_dataset = data_ontime.join(data_sum, rsuffix='_r')
merged_dataset = merged_dataset.drop(['AirLine_r'], axis=1)

merged_dataset['performance'] = merged_dataset.apply(lambda x: x[2]*100/x[3], axis=1)

merged_dataset.sort_values(by='performance', ascending=False)

,AirLine,Status,ontimeCount,sum,performance
1,AMWEST,onTime,6438,7225,89.107266
0,ALASKA,onTime,3274,3775,86.728477


### Question 4
 We can make the initial dataset even more wider by setting the status(delayed/ontime) as columns. Then the columns will be as bellow-
* onTime-LosAngeles, onTime-Phoenix, onTime-SanDiego, onTime-SanFrancisco, onTime-Seattle, 
* delayed-LosAngeles, delayed-Phoenix, delayed-SanDiego, delayed-SanFrancisco, delayed-Seattle